### This notebook contains simulations for constructing and evaluating the max-entropy distributions for large version of Compas dataset

The code is based on the following paper:

**Data preprocessing to mitigate bias: A maximum-entropy based approach** <br>
L.Elisa Celis, Vijay Keswani, Nisheeth K. Vishnoi <br>
ICML 2020

In [1]:
%load_ext autoreload
%autoreload 2

#This project requires the IBM AIF360 package for the datasets (https://github.com/ibm/aif360)

import numpy as np
from FairMaxEnt.domain import Domain
from FairMaxEnt.memory import MemoryTrie
from FairMaxEnt.maximum_entropy_distribution import MaxEnt
from FairMaxEnt.fair_maximum_entropy import FairMaximumEntropy
from FairMaxEnt.fair_maximum_entropy import reweightSamples
from Codes.Utils import *

from tqdm import tqdm_notebook as tqdm


#### Loading the Compas dataset

In [5]:
simpleDomain, simpleSamples = getLargeCompasDataset()
simpleDomain, len(simpleSamples)

8723
['Other', 'Asian', 'Hispanic', 'Caucasian', 'Native American', 'African-American']


(Domain in 19 with 19, 8723)

In [6]:
simpleDomain.labels

['sex',
 'age',
 'race',
 'juv_fel_count',
 'decile_score',
 'juv_misd_count',
 'juv_other_count',
 'priors_count',
 'days_in_jail',
 'c_charge_degree',
 'is_violent',
 'is_drug',
 'is_firearm',
 'is_MinorInvolved',
 'is_roadSafetyHazard',
 'is_sexoffense',
 'is_fraud',
 'is_petty',
 'is_recid']

#### Example runs of max-entropy optimization program

In [7]:
# The setup is done and now we can run the experiments
# C - smoothing parameter
# delta - error parameter

C = 0.1
delta = 0

sens_attr = 2   # for Compas
# sens_attr = 1  # for Adult

# labelIndex denotes the index of class label
labelIndex = len(simpleSamples[0]) - 1

The fairness metrics, evaluated over the original raw dataset, have the following values

In [63]:
def getDisparateImpact(dat, sens_attr):
    labelIndex = len(dat[0])-1
    dataset = []
    for row in dat:
        if row[sens_attr] == -1:
            continue
        dataset.append(row)
    
    dataset = np.array(dataset)
    y1z1 = sum(dataset[:,-1] * dataset[:,sens_attr])
    y1z0 = sum(dataset[:,-1] * (1-dataset[:,sens_attr]))
    z1 = sum(dataset[:,sens_attr])
    z0 = len(dataset) - z1
    print (y1z0, y1z1)
    print (z1/z0)
    return min((y1z0/z0)/(y1z1/z1), (y1z1/z1)/(y1z0/z0)) 


In [9]:
print("Statistical Rate: ", getDisparateImpact(simpleSamples, sens_attr))
print("Representation rate: ", getGenderRatio(simpleSamples, sens_attr))

Statistical Rate:  0.7660012166325124
Representation rate:  0.556566738044254


Since we also need to calculate utility from empirical distribution of original dataset, we first find the way of calculating the utility

In [11]:
# The utility evaluation procedure for large COMPAS dataset is different from the one for small dataset.

rawCorr = getCorr(simpleSamples)
getCorrUtility(simpleSamples, rawCorr)

0.0

#### We first look at max-entropy distribution using dataset mean and prior
The prior in this case is $q_C^d$ and the expected value is mean of original dataset, $\theta^d$

In [10]:
maxEnt = FairMaximumEntropy(simpleDomain, simpleSamples, C, delta, labelIndex, reweight=False, weightedMean=False)
dataset = maxEnt.sample(10000)

print("Statistical Rate: ", getDisparateImpact(dataset, sens_attr))
print("Representation rate: ", getGenderRatio(dataset, sens_attr))
print("Covariance matrix difference norm: ", getCorrUtility(dataset, rawCorr))

/usr/local/lib/python3.7/site-packages/theano/tensor/basic.py:6105: UserWarning: Warning, Cannot compute test value: input 1 (lambda) of Op dot(<TensorType(float64, matrix)>, lambda) missing default value
  return _dot(a, b)
/Users/vijaykeswani/FairMaxEnt/FairMaxEnt Code/FairMaxEnt/counting_oracle.py:48: UserWarning: Warning, Cannot compute test value: input 0 (dot.0) of Op Elemwise{exp,no_inplace}(dot.0) missing default value
  return Prior*T.exp(T.dot(Support, Lambda))
/usr/local/lib/python3.7/site-packages/theano/tensor/var.py:155: UserWarning: Warning, Cannot compute test value: input 1 (Elemwise{exp,no_inplace}.0) of Op Elemwise{mul,no_inplace}(<TensorType(float64, vector)>, Elemwise{exp,no_inplace}.0) missing default value
  return theano.tensor.mul(self, other)
/usr/local/lib/python3.7/site-packages/theano/tensor/basic.py:3210: UserWarning: Warning, Cannot compute test value: input 0 (Elemwise{mul,no_inplace}.0) of Op Sum{acc_dtype=float64}(Elemwise{mul,no_inplace}.0) missing de

/usr/local/lib/python3.7/site-packages/theano/tensor/elemwise.py:635: UserWarning: Warning, Cannot compute test value: input 0 (Elemwise{neg}.0) of Op Elemwise{true_div}(Elemwise{neg}.0, Elemwise{mul}.0) missing default value
  *[transform(ipt) for ipt in node.inputs])
/usr/local/lib/python3.7/site-packages/theano/tensor/var.py:275: UserWarning: Warning, Cannot compute test value: input 0 (Elemwise{true_div}.0) of Op Shape(Elemwise{true_div}.0) missing default value
  shape = property(lambda self: theano.tensor.basic.shape(self))
/usr/local/lib/python3.7/site-packages/theano/gof/op.py:1109: UserWarning: Argument 0 has no test value
  " has no test value")
/usr/local/lib/python3.7/site-packages/theano/gof/op.py:1109: UserWarning: Argument 1 has no test value
  " has no test value")
/usr/local/lib/python3.7/site-packages/theano/tensor/var.py:275: UserWarning: Warning, Cannot compute test value: input 0 (Sum{acc_dtype=float64}.0) of Op Shape(Sum{acc_dtype=float64}.0) missing default value

/usr/local/lib/python3.7/site-packages/theano/tensor/elemwise.py:635: UserWarning: Warning, Cannot compute test value: input 0 (Elemwise{neg}.0) of Op Elemwise{true_div}(Elemwise{neg}.0, Elemwise{mul}.0) missing default value
  *[transform(ipt) for ipt in node.inputs])
/usr/local/lib/python3.7/site-packages/theano/tensor/var.py:275: UserWarning: Warning, Cannot compute test value: input 0 (Elemwise{true_div}.0) of Op Shape(Elemwise{true_div}.0) missing default value
  shape = property(lambda self: theano.tensor.basic.shape(self))
/usr/local/lib/python3.7/site-packages/theano/gof/op.py:1109: UserWarning: Argument 0 has no test value
  " has no test value")
/usr/local/lib/python3.7/site-packages/theano/gof/op.py:1109: UserWarning: Argument 1 has no test value
  " has no test value")
/usr/local/lib/python3.7/site-packages/theano/tensor/var.py:275: UserWarning: Warning, Cannot compute test value: input 0 (Elemwise{exp,no_inplace}.0) of Op Shape(Elemwise{exp,no_inplace}.0) missing default v

/usr/local/lib/python3.7/site-packages/theano/tensor/elemwise.py:635: UserWarning: Warning, Cannot compute test value: input 0 (Elemwise{mul}.0) of Op Elemwise{true_div}(Elemwise{mul}.0, <TensorType(int64, scalar)>) missing default value
  *[transform(ipt) for ipt in node.inputs])
/usr/local/lib/python3.7/site-packages/theano/tensor/elemwise.py:635: UserWarning: Warning, Cannot compute test value: input 0 (Elemwise{mul}.0) of Op Elemwise{mul}(Elemwise{mul}.0, Elemwise{exp,no_inplace}.0) missing default value
  *[transform(ipt) for ipt in node.inputs])
/usr/local/lib/python3.7/site-packages/theano/tensor/elemwise.py:635: UserWarning: Warning, Cannot compute test value: input 1 (Elemwise{exp,no_inplace}.0) of Op Elemwise{mul}(Elemwise{mul}.0, Elemwise{exp,no_inplace}.0) missing default value
  *[transform(ipt) for ipt in node.inputs])
/usr/local/lib/python3.7/site-packages/theano/tensor/basic.py:5620: UserWarning: Warning, Cannot compute test value: input 1 (Subtensor{int64}.0) of Op ARa

/usr/local/lib/python3.7/site-packages/theano/tensor/basic.py:1647: UserWarning: Warning, Cannot compute test value: input 0 (Sum{axis=[1], acc_dtype=float64}.0) of Op InplaceDimShuffle{0,x}(Sum{axis=[1], acc_dtype=float64}.0) missing default value
  return DimShuffle(y.type.broadcastable, new_dims)(y)
/usr/local/lib/python3.7/site-packages/theano/tensor/var.py:275: UserWarning: Warning, Cannot compute test value: input 0 (InplaceDimShuffle{0,x}.0) of Op Shape(InplaceDimShuffle{0,x}.0) missing default value
  shape = property(lambda self: theano.tensor.basic.shape(self))
/usr/local/lib/python3.7/site-packages/theano/tensor/elemwise.py:306: UserWarning: Warning, Cannot compute test value: input 0 (InplaceDimShuffle{0,x}.0) of Op Elemwise{identity}(InplaceDimShuffle{0,x}.0) missing default value
  Elemwise(scalar.identity)(gz))]
/usr/local/lib/python3.7/site-packages/theano/tensor/elemwise.py:635: UserWarning: Warning, Cannot compute test value: input 0 (InplaceDimShuffle{0}.0) of Op Ele

/usr/local/lib/python3.7/site-packages/theano/tensor/var.py:128: UserWarning: Warning, Cannot compute test value: input 0 (Elemwise{add,no_inplace}.0) of Op Elemwise{add,no_inplace}(Elemwise{add,no_inplace}.0, Elemwise{mul}.0) missing default value
  return theano.tensor.basic.add(self, other)
/usr/local/lib/python3.7/site-packages/theano/tensor/var.py:128: UserWarning: Warning, Cannot compute test value: input 1 (Elemwise{mul}.0) of Op Elemwise{add,no_inplace}(Elemwise{add,no_inplace}.0, Elemwise{mul}.0) missing default value
  return theano.tensor.basic.add(self, other)
/usr/local/lib/python3.7/site-packages/theano/scan_module/scan.py:1077: UserWarning: Warning, Cannot compute test value: input 0 (Subtensor{int64}.0) of Op for{cpu,scan_fn}(Subtensor{int64}.0, Subtensor{:int64:}.0, Subtensor{int64}.0, <TensorType(float64, matrix)>, <TensorType(float64, vector)>, <TensorType(float64, scalar)>, <TensorType(float64, 3D)>, <TensorType(float64, matrix)>, <TensorType(int64, scalar)>, <Tenso

Statistical Rate:  0.7592239314851646
Gender Ratio:  0.5503875968992248


NameError: name 'rawCorr' is not defined

The raw dataset, in this case, is quite biased. Hence using just $q_C^d$ and $\theta^d$ will not lead to a fair max-entropy distribution

#### Next we can set different prior and expected values to debias the distribution

The parameter *reweight* can be set true if the re-weighted prior distribution (i.e., $q_C^w$) should be used.
The parameter *weightedMean* can be set true if the re-weighted expected value should be used (i.e., $\theta^w$).
Using these, we get a max-entropy distribution that has high statistical and representation rate.

In [13]:
maxEnt = FairMaximumEntropy(simpleDomain, simpleSamples, C, delta, sens_attr,
                                reweight=True, reweightXindices=[sens_attr],
                                reweightYindices=[len(simpleSamples[0])-1], weightedMean=True)
dataset = maxEnt.sample(10000)

print("Statistical Rate: ", getDisparateImpact(dataset, sens_attr))
print("Representation rate: ", getGenderRatio(dataset, sens_attr))
print("Covariance matrix difference norm: ", getCorrUtility(dataset, rawCorr))

Statistical Rate:  0.987810898133266
Representation rate:  0.9657951641438962
Covariance matrix difference norm:  1.2665812336846956


Next we *reweight* to be true, but *weightedMean* to be false, i.e., using fair prior but the expected value is the mean of the original dataset. With this combination, we get a distribution with high statistical rate but low representation rate.

In [14]:
labelIndex = len(simpleSamples[0])-1
maxEnt = FairMaximumEntropy(simpleDomain, simpleSamples, C, delta, 0,
                            reweight=True, reweightXindices=[0],
                            reweightYindices=[labelIndex])

dataset = maxEnt.sample(10000)
print("Statistical Rate: ", getDisparateImpact(dataset, sens_attr))
print("Representation rate: ", getGenderRatio(dataset, sens_attr))
print("Covariance matrix difference norm: ", getCorrUtility(dataset, rawCorr))

Statistical Rate:  0.8022132230728838
Representation rate:  0.5401201293700909
Covariance matrix difference norm:  0.9932932469965341


To use $\theta^b$, we need to set *alterMean* to be true. With this combination, we again get a distribution with high statistical rate and high representation rate.

In [22]:
%%time
# The parameter alterMean can be set true if the balanced expected value should be used


maxEnt = FairMaximumEntropy(simpleDomain, simpleSamples, C, delta, 0,
                                reweight=True, reweightXindices=[0],
                                reweightYindices=[labelIndex], alterMean = True)

dataset = maxEnt.sample(10000)
print("Statistical Rate: ", getDisparateImpact(dataset, sens_attr))
print("Representation rate: ", getGenderRatio(dataset, sens_attr))
print("Covariance matrix difference norm: ", getCorrUtility(dataset, rawCorr))

Statistical Rate:  0.9738431737582841
Gender Ratio:  0.9688915140775743
Covariance matrix difference norm:  2.0037413244353943
CPU times: user 42.5 s, sys: 585 ms, total: 43.1 s
Wall time: 43.9 s


#### Experiments in the paper
In the paper, we report value after 5-fold cross-validation. For each fold, we do 100 simulations.
We also provide results for different values of C. 

The max-entropy distribution computed below represent all possible choices of prior distribution, expected vector and parameter $C$.

In [15]:
maxEnts = {'maxEnt_unif_wt_unif_mean' : {}, 
                  'maxEnt_re_wt_unif_mean' : {}, 
                  'maxEnt_unif_wt_alt_mean' : {}, 
                  'maxEnt_re_wt_alt_mean' : {}, 
                  'maxEnt_unif_wt_wt_mean' : {}, 
                  'maxEnt_re_wt_wt_mean' : {}}

for key in maxEnts.keys():
    for D in range(11):
        C = D/10.0
        maxEnts[key][C] = {}
        

delta = 0
labelIndex = len(simpleSamples[0]) - 1

for fold in tqdm(range(5)):
    
    trainData, testData = getTrainAndTestData(simpleSamples, fold)
    
    for D in [5]:
        C = D/10.0
        key = "{fold}_{C}".format(fold=fold, C=C)
        
        
        maxEnts['maxEnt_unif_wt_unif_mean'][C][fold] = FairMaximumEntropy(simpleDomain, trainData, 
                                          C, 
                                          delta, 
                                          sens_attr)
        
        
        maxEnts['maxEnt_re_wt_unif_mean'][C][fold] = FairMaximumEntropy(simpleDomain, 
                                          trainData, 
                                          C, 
                                          delta, 
                                          sens_attr,
                                          reweight=True, 
                                          reweightXindices=[sens_attr],
                                          reweightYindices=[labelIndex])


        maxEnts['maxEnt_unif_wt_alt_mean'][C][fold] = FairMaximumEntropy(simpleDomain, 
                                          trainData, 
                                          C, 
                                          delta, 
                                          sens_attr,
                                          alterMean = True)

        
        maxEnts['maxEnt_re_wt_alt_mean'][C][fold] = FairMaximumEntropy(simpleDomain, trainData, C, delta, sens_attr,
                                reweight=True, reweightXindices=[sens_attr],
                                reweightYindices=[labelIndex], alterMean = True)

        
        maxEnts['maxEnt_unif_wt_wt_mean'][C][fold] = FairMaximumEntropy(simpleDomain, trainData, C, delta, sens_attr,
                                weightedMean=True, reweightXindices=[sens_attr],
                                reweightYindices=[labelIndex])

            
        maxEnts['maxEnt_re_wt_wt_mean'][C][fold] = FairMaximumEntropy(simpleDomain, trainData, C, delta, sens_attr,
                                reweight=True, reweightXindices=[sens_attr],
                                reweightYindices=[labelIndex], weightedMean=True)
    
        

In [17]:
DIs, KLs, GRs, Accs, CDs = [], [], [], [], []
for key in tqdm(["maxEnt_re_wt_wt_mean","maxEnt_re_wt_alt_mean"]):
    diForKey, klForKey, grForKey, accForKey, cdForKey = [], [], [], [], []
    for D in [5]:
        C = D/10.0
        for fold in tqdm(range(5)):
            di, kl, gr, acc, cd = [] ,[], [], [], []
            for _ in tqdm(range(10)):
                dataset = maxEnts[key][C][fold].sample(10000)
            
                di.append(getDisparateImpact(dataset, sens_attr))
                kl.append(getCorrUtility(dataset, rawCorr))
                gr.append(getGenderRatio(dataset, sens_attr))
                
                _, testData = getTrainAndTestData(simpleSamples, fold)    
                a1, cd1 = getClfAccAndDI(dataset, testData, sens_attr, clf = DecisionTreeClassifier(random_state=0))
                acc.append(a1)
                cd.append(cd1)
            
            diForKey = diForKey + di
            klForKey = klForKey + kl
            grForKey = grForKey + gr
            accForKey = accForKey + acc
            cdForKey = cdForKey + cd
    DIs.append(diForKey)
    KLs.append(klForKey)
    GRs.append(grForKey)
    Accs.append(accForKey)
    CDs.append(cdForKey)
            
        


 key
0.9689046495810351 0.017444228364868163
0.9850448997287535 0.011514742905800728
0.906066545426816 0.045232557392931895
1.8696825202409548 0.25896184273613
0.6326747622398554 0.012792654968900224



 key
0.9753125299386507 0.02003200266892549
0.9854205438220263 0.010574573724882482
0.9087360761802672 0.05600147384839257
1.9331732477050638 0.2405877667738275
0.6349912563083241 0.012899651095349116

